# Dependencies & Read Data

In [1]:
#Imports
import pandas as pd
import requests
import gmaps
import re
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [42]:
#Read the CSV & Create DF
clean_geocoded_df = pd.read_csv("cleaned_facility_data.csv")
clean_geocoded_df.head(1)

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,origin_lat,origin_long,google_lat,google_long
0,FA0003323,RUNZA,1743 MAIN ST LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,40.189746,-105.102332,40.189772,-105.102667


# Assemble Places API Call 

In [55]:
#Configure API Call for Nearby Search using enough parameters to ensure we get the right place! 
params = {
    "rankby": "distance",
    "keyword": "restaurant",
    "location": "40.189772, -105.102667",
    "name": "RUNZA",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

rest = requests.get(base_url, params=params).json()

rest["results"][0]["price_level"]

1

# Make the Call

In [57]:
params = {
    "rankby": "distance",
    "keyword": "restaurant",
    "key": g_key
}

#iterate through that dataframe!
for index, row in clean_geocoded_df.iterrows():
    try: 
        # Get the Name Address from the dataframe
        name = row["facilityName"]
        lat = row["google_lat"]
        long = row["google_long"]
        params["location"] = f"{lat},{long}"
        params["name"] = name

        # Set the base URL to grab JSON data
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # Make request and retrieve the JSON data from the search. 
        search = requests.get(base_url, params=params).json()

        #Set Biz status, Name, Rating, price_level, user_ratings_total variables
        g_status = search["results"][0]["business_status"] 
        g_name = search["results"][0]["name"]
        g_rating = search["results"][0]["rating"]
        g_price_level = search["results"][0]["price_level"]
        g_user_ratings = search["results"][0]["user_ratings_total"]

        # Get all fields and store in new columns
        clean_geocoded_df.loc[index, "biz_status"] = g_status
        clean_geocoded_df.loc[index, "gbiz_name"] = g_name
        clean_geocoded_df.loc[index, "rating"] = g_rating
        clean_geocoded_df.loc[index, "price_level"] = g_price_level
        clean_geocoded_df.loc[index, "total_user_ratings"] = g_user_ratings
               
    except (KeyError):
        print(f"{name} not found, skip - Key") 
    except (IndexError):
         print(f"{name} not found, skip - Index") 
clean_geocoded_df.head()

ROASTED TOAD BBQ THE not found, skip - Key
FAIR ISLE COFFEE CO @ LGMT FARM MRKT not found, skip - Index
CARNICERIA EL CERRITO not found, skip - Key
UTE CREEK GOLF COURSE not found, skip - Key
LEVY IPF 1 (MOBILE) not found, skip - Key
BOULDER BEEF (MOBILE PRE-PKG) not found, skip - Index
MOUNTAIN VIEW GRILL 2  (MOBILE) not found, skip - Index
CUPPA HOUSE COFFEE & TEA not found, skip - Key
(CU) C4C THE BAKERY not found, skip - Key
AREPA JOINT THE (MOBILE) not found, skip - Key
NIWOT WHEEL WORKS not found, skip - Key
ARAPAHOE MEAT CO not found, skip - Key
CHOP 5 SALAD KTICHEN not found, skip - Key
SMOKIN BOWLS not found, skip - Key
(CU)  FARRAND ALLEY not found, skip - Key
EGGLOO DELIGHTS (SPECIAL EVENT) not found, skip - Index
LA PUPUSA LOCA (MOBILE) not found, skip - Key
LA FAMILIA (MOBILE) not found, skip - Key
FRASIER MEADOWS MANOR KITCHEN not found, skip - Key
HOMEWOOD SUITES BOULDER not found, skip - Key
GREAT OUTDOORS WATER PARK THE not found, skip - Key
MEEKER PARK LODGE not found

DOS WAPOS not found, skip - Key
COOL BEADS (MOBILE) not found, skip - Index
JFE SUSHI (KS-013) not found, skip - Key
WALGREENS #19103 not found, skip - Index
SENOR GOMEZ not found, skip - Key
ELDORA MTN RESORT LOOKOUT RESTAURANT not found, skip - Key
JESTERS PERFORMING ARTS not found, skip - Key
SUR LA TABLE not found, skip - Key
AMERICAN LEGION POST 111 not found, skip - Key
LAS CAZUELAS not found, skip - Key
WILD BASIN LODGE & EVENT CENTER not found, skip - Index
GARDEN GATE CAFE THE not found, skip - Key
RAYBACK COLLECTIVE not found, skip - Key
NATURAL GROCERS BY VITAMIN COTTAGE not found, skip - Key
COURTYARD BY MARRIOTT not found, skip - Key
STARBUCKS 21402 not found, skip - Index
EUREST DINING SERVICE AT IBM CANTEE not found, skip - Key
SPRUCE CAFE not found, skip - Key
NCAR not found, skip - Key
LONGMONT YOUTH CENTER not found, skip - Key
CURE ORGANIC FARM (FARMERS MARKET) not found, skip - Key
COLORADO ATHLETIC CLUB not found, skip - Key
7 ELEVEN 41867 not found, skip - Index
K

,facilityId,facilityName,concat_address,typeOfFacility,categoryOfFacility,origin_lat,origin_long,google_lat,google_long,biz_status,gbiz_name,rating,price_level,total_user_ratings
0,FA0003323,RUNZA,1743 MAIN ST LONGMONT CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,40.189746,-105.102332,40.189772,-105.102667,OPERATIONAL,Runza Restaurant,4.2,1.0,210.0
1,FA0000616,FRANKS CHOP HOUSE,921 WALNUT ST BOULDER CO USA,RESTAURANT MORE THAN 200 SEATS,FULL SERVICE FULL MENU,40.016235,-105.282759,40.016553,-105.282845,OPERATIONAL,Frank’s Chophouse,4.3,3.0,446.0
2,FA0004494,ROASTED TOAD BBQ THE,229 HWY 119 NEDERLAND CO USA,RESTAURANT 0 TO 100 SEATS,FULL MENU LIMITED SERVICE,NaN,NaN,39.963180,-105.509594,NaN,NaN,NaN,NaN,NaN
3,FA0003893,FAIR ISLE COFFEE CO @ LGMT FARM MRKT,237 COLLYER ST Longmont CO USA,SPECIAL EVENT,SPECIAL EVENT,40.163078,-105.098175,40.163077,-105.098557,NaN,NaN,NaN,NaN,NaN
4,FA0003472,TODS ESPRESSO CAFE,6558 LOOKOUT RD BOULDER CO USA,LIMITED FOOD SERVICE CONVENIENCE OTHER,FAST FOOD LIMITED MENU,40.072805,-105.201040,40.072171,-105.200873,OPERATIONAL,Tod's Espresso Cafe,4.3,1.0,170.0


# Export Data

In [60]:
# Export unique address dataset w/ Google Data
output_data = "facility_data_with_ratings.csv"
clean_geocoded_df.to_csv(output_data,header=True,index=False)